# Classification: Klassische Methoden
In diesem Notebook versuchen wir die Klassifizierung in "Failure"/"No Failure" mit klassischen machine learning Methoden durchzuführen.

In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn import svm
from sklearn import metrics
import Preprocessing as pp
import Helpers

path_data = '/Users/marvinwoller/Desktop/SmartDataAnalytics/Blatt2/data/'

rootdir_train = path_data + 'train/'
rootdir_test = path_data + 'test/'

train_labels_path = path_data + 'train_label.csv'
test_labels_path = path_data + 'test_label.csv'

feature_path = path_data + 'features/'
feature_path_test = path_data + 'features_test/'

# Merge labels and data
train_labels = pd.read_csv(train_labels_path, index_col=0) #Don't use index numbers per row but CSV file name as index

In [11]:
# Read feature data (train)
df = pd.read_csv(feature_path + 'median' + '.csv', index_col=0)

In [3]:
# Preprocessing (train)
# Select Features
df = pp.drop_columns_with_drift(df, intensity='light')
df.info()

# Remove Outliers
df = pp.remove_outliers(df)

df_random = df.sample(n=3000, random_state=123) # Select random sample because dataset is too large for some classifiers

# Get lables from dataframe
labels = df['label']
labels_random = df_random['label']
df.drop(['area','label'], axis = 1, inplace=True)
df_random.drop(['area','label'], axis = 1, inplace=True)

# Scaling
df = pp. perform_scaling(df)
df_random = pp.perform_scaling(df_random)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33360 entries, 0 to 33359
Data columns (total 22 columns):
Wheel speed                               33360 non-null float64
hub angle                                 33360 non-null float64
blade 1 angle                             33360 non-null float64
blade 2 angle                             33360 non-null float64
blade 3 angle                             33360 non-null float64
overspeed sensor speed detection value    33360 non-null float64
5 second yaw against wind average         33360 non-null float64
Aircraft weather station wind speed       33360 non-null float64
wind direction absolute value             33360 non-null float64
atmospheric pressure                      33360 non-null float64
reactive power control status             33360 non-null float64
inverter grid side voltage                33360 non-null float64
inverter grid side reactive power         33360 non-null float64
generator torque                          3336

In [16]:
# Read and preprocess test feature data
df_test = pd.read_csv(feature_path_test + 'median' + '.csv', index_col=0)
print(df_test.info())
df_test = pp.drop_columns_with_drift(df_test, intensity='light')
print(df_test.info())
df_test = pp.remove_outliers(df_test)
print(df_test.info())
labels = df_test['label']
df_test.drop(['area','label'], axis = 1, inplace=True)
df_test = pp.perform_scaling(df_test)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16286 entries, 0 to 16285
Data columns (total 74 columns):
Wheel speed                               16286 non-null float64
hub angle                                 16286 non-null float64
blade 1 angle                             16286 non-null float64
blade 2 angle                             16286 non-null float64
blade 3 angle                             16286 non-null float64
pitch motor 1 current                     16286 non-null float64
pitch motor 2 current                     16286 non-null float64
Pitch motor 3 current                     16286 non-null float64
overspeed sensor speed detection value    16286 non-null float64
5 second yaw against wind average         16286 non-null float64
x direction vibration value               16286 non-null float64
y direction vibration value               16286 non-null float64
hydraulic brake pressure                  16286 non-null float64
Aircraft weather station wind speed       1628

/Users/marvinwoller/Desktop/SmartDataAnalytics_Gruppe1/Übungsblatt2/Preprocessing.py:27: RuntimeWarning: invalid value encountered in less
  df_local = df_local[(z < 3).all(axis=1)]


ValueError: Found array with 0 sample(s) (shape=(0, 22)) while a minimum of 1 is required by MinMaxScaler.

In [6]:
# ################################## Classificaton with basic methods ###############################
# Classification: SVM 
X_train, X_test, y_train, y_test = train_test_split(df[0:5000], labels[0:5000], test_size=0.2, random_state=123)

In [7]:
#Create a svm Classifier
#clf = svm.SVC(kernel='linear', gamma='auto', verbose=1) # Linear Kernel
clf = BaggingClassifier(base_estimator=SVC(kernel='linear', gamma='auto'), n_estimators=10, random_state=0, n_jobs=-1)

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset (here still part of training dataset to avoid concept drift)
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))



# Classification: Lineares Modell

KeyboardInterrupt: 

In [ ]:
# Classification: Logitisches Modell
#Logistic Regression with sklearn
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)
print(logreg.score(X_test,y_test))

# Classification: KNN

# Classification: Xgboost

# Classification: LightGBM

# Others ?

